# HOME ASSIGNMENT #4: DATABASE DESIGN & SQL

**Mục đích của bài Assignment**
> *   Hiểu các bước design Database cho Case study cụ thể: DataCracy
*   Bài tập SQL (Dựa trên Cheatsheet)
*   `[Optional]` Bài tập Python Pandas (Dựa trên Cheatsheet)

**Các kiến thức áp dụng**
* Slack API, JSON to DataFrame
* Phân tích Data (Assignment#1, Lab#1)
* Database (DB) Design
* SQL
* Python Pandas

**Lời Khuyên**
* Đây là bài tập dài, nhưng thiên về hiểu case studies và thiết kế => Nên bạn **hãy bắt đầu sớm**
* **Chắc chắn là bạn đọc thực kỹ các bước hướng dẫn và tài liệu, tận dụng Slack để trao đổi**
* **Đừng sa đà vào Code của STEP 0** (bước này tương tự Assignmetn 3 về Slack API, nhưng biến đổi thêm cho phù hợp)
* **Đừng sa đà vào các tiểu tiết** => Rất quan trọng là bạn cố gắng đi đến hết `TODO#6` để hiểu bức tranh toàn cảnh và kết nối được kiến thức. Sau khi đi hết qua, các bạn có thể trở lại và nhìn cận vào các tiểu tiết
* Làm nhiều nhất có thể (Đừng quá lo lắng nếu bạn không thể hoàn thành hết)

In [4]:
import requests # -> Để call API
import json # -> Xử lý file JSON
import pandas as pd # -> Thư viện xử lý dữ liệu dạng bảng
import numpy as np
import re # -> Thư viện xử lý text: regular expressions
from datetime import datetime as dt # -> Thư viện xử lý dữ liệu thời gian
import duckdb # -> Thư viện "giả lập" xử lý dữ liệu bằng SQL 

## STEP 0: XEM LẠI DATA ĐÃ CÓ (SLACK API)

Ở bước đầu tiên, ta xem lại tất cả các data của DATACracy Atom mà ra đã biết: 
* **Data từ Slack API**:
  * Danh sách thành viên
  * Danh sách các channels
  * Lịch sử tin nhắn trên các channels
* **Data do dự án tự collect - File CSV (trích xuất từ Google Spreadsheet)**:
  * Danh sách thành viên được phân theo vị trí (mentors, learners, BTC)


===> Các dữ liệu này lần lượt được lấy bằng code bên dưới.


In [5]:
## Load Token file 
## WARNING!! --- Put it in gitignore and DO NOT print out to notebook
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)

### 0.1. Pull List of Members

In [7]:
# 1. LIST OF SLACK MEMBERS 

## Pull list of member as JSON
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(json_data['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

## Convert to CSV
user_dict = {'user_id':[],'name':[],'display_name':[],'real_name':[],'title':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

user_df = pd.DataFrame(user_dict) 
user_df = user_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
user_df.head()

,user_id,name,display_name,real_name,title,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,NaN,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,NaN,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,NaN,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,NaN,False


### 0.2. List of Channels

In [8]:
# 2. LIST OF SLACK CHANNELS

endpoint2 = "https://slack.com/api/conversations.list"
data = {'types': 'public_channel,private_channel'} # -> CHECK: API Docs https://api.slack.com/methods/conversations.list/test
response_json = requests.post(endpoint2, headers=headers, data=data).json() 
channel_dat = response_json['channels']

channel_dict = {'channel_id':[], 'channel_name':[], 'is_channel':[],'creator':[],'created_at':[],'topics':[],'purpose':[],'num_members':[]}
for i in range(len(channel_dat)):
  channel_dict['channel_id'].append(channel_dat[i]['id'])
  channel_dict['channel_name'].append(channel_dat[i]['name'])
  channel_dict['is_channel'].append(channel_dat[i]['is_channel'])
  channel_dict['creator'].append(channel_dat[i]['creator'])
  channel_dict['created_at'].append(dt.fromtimestamp(float(channel_dat[i]['created'])))
  channel_dict['topics'].append(channel_dat[i]['topic']['value'])
  channel_dict['purpose'].append(channel_dat[i]['purpose']['value'])
  channel_dict['num_members'].append(channel_dat[i]['num_members'])

channel_df = pd.DataFrame(channel_dict) 
channel_df = channel_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
channel_df.head()

,channel_id,channel_name,is_channel,creator,created_at,topics,purpose,num_members
0,C01B4PVGLVB,general,True,U01BE2PR6LU,2020-09-23 17:25:03,NaN,This is the one channel that will always inclu...,65
1,C01BYH7JHB5,contents,True,U01BE2PR6LU,2020-10-05 14:28:09,NaN,NaN,12
2,C01CAMNCJJV,branding-design,True,U01AVDY7JET,2020-10-05 10:16:30,NaN,NaN,9
3,C01U6P7LZ8F,atom-assignment1,True,U01BE2PR6LU,2021-04-17 14:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44
4,C01UL6K1C7L,atom-week1,True,U01BE2PR6LU,2021-04-17 00:44:32,TOPIC: Data Strategy. Metrics Design. Spreadsh...,TOPIC: Data Strategy. Metrics Design. Spreadsh...,43


### 0.3. Message Data

In [9]:
endpoint3 = "https://slack.com/api/conversations.history"

In [10]:
msg_dict = {'channel_id':[],'msg_id':[], 'msg_ts':[], 'user_id':[], 'latest_reply':[],'reply_user_count':[],'reply_users':[],'github_link':[]}
for channel_id, channel_name in zip(channel_df['channel_id'], channel_df['channel_name']):
  print('Channel ID: {} - Channel Name: {}'.format(channel_id, channel_name))
  try:
    data = {"channel": channel_id} 
    response_json = requests.post(endpoint3, data=data, headers=headers).json()
    msg_ls = response_json['messages']
    for i in range(len(msg_ls)):
      if 'client_msg_id' in msg_ls[i].keys():
        msg_dict['channel_id'].append(channel_id)
        msg_dict['msg_id'].append(msg_ls[i]['client_msg_id'])
        msg_dict['msg_ts'].append(dt.fromtimestamp(float(msg_ls[i]['ts'])))
        msg_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'] if 'latest_reply' in msg_ls[i].keys() else 0))) ## -> No reply: 1970-01-01
        msg_dict['user_id'].append(msg_ls[i]['user'])
        msg_dict['reply_user_count'].append(msg_ls[i]['reply_users_count'] if 'reply_users_count' in msg_ls[i].keys() else 0)
        msg_dict['reply_users'].append(msg_ls[i]['reply_users'] if 'reply_users' in msg_ls[i].keys() else 0) 
        ## -> Censor message contains tokens
        text = msg_ls[i]['text']
        github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text)
        msg_dict['github_link'].append(github_link[0] if len(github_link) > 0 else np.nan)
  except:
    print('====> '+ str(response_json))

Channel ID: C01B4PVGLVB - Channel Name: general
Channel ID: C01BYH7JHB5 - Channel Name: contents
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01CAMNCJJV - Channel Name: branding-design
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01U6P7LZ8F - Channel Name: atom-assignment1
Channel ID: C01UL6K1C7L - Channel Name: atom-week1
Channel ID: C01ULCHGN75 - Channel Name: atom-general
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C020VMT58JK - Channel Name: topics-data-analytics
Channel ID: C0213MNH9L6 - Channel Name: topics-python
Channel ID: C0213N56M2A - Channel Name: topics-materials
Channel ID: C021FSDN7LJ - Channel Name: atom-assignment2
Channel ID: C021KLB0DSB - Channel Name: discuss-group3
Channel ID: C021KLB90GP - Channel Name: discuss-group4
Channel ID: C02204B2CD6 - Channel Name: atom-week2
Channel ID: C0220KU9399 - Channel Name: discuss-group1
Channel ID: C0226D3LEQ4 - Channel Name: atom-week3
Channel ID: C0227A51SAY - Channel Name: atom-ass

In [11]:
msg_df = pd.DataFrame(msg_dict)
msg_df = msg_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
msg_df.tail()

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link
261,C022RRWQ6US,3db33d2a-5666-44cf-95e0-dd33aee4e63c,2021-05-24 18:49:29.003700,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,0,NaN
262,C022RRWQ6US,8b4a2ebb-7882-402b-8339-a0b067ce5165,2021-05-23 14:58:23.003000,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,0,NaN
263,C022RRWQ6US,231fad01-7083-4cd0-b6b6-e97bf54779e2,2021-05-23 06:23:13.001400,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,0,https://github.com/anhdanggit/
264,C022Y1FUETE,ee582f3d-c339-4d06-a263-b71c6470e647,2021-05-24 18:49:57.001000,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,0,NaN
265,C022Y1FUETE,38adf48f-21cc-437c-9ad3-31cd002bc4f8,2021-05-23 14:58:05.001500,U01BE2PR6LU,2021-05-23 23:15:45.002200,1,[U01VB632FPS],NaN


### 0.4. DataCracy Info
* **Data do dự án tự collect - File CSV (trích xuất từ Google Spreadsheet)**: Danh sách thành viên được phân theo vị trí (mentors, learners, BTC)
* Trong cùng folder Github `assignment_4`

In [12]:
dtc_groups = pd.read_csv('datacracy_groups.csv')
dtc_groups.head()

,name,DataCracy_role
0,slackbot,BTC
1,loclexuan26392,BTC
2,locle.ds,BTC
3,maianhdang.ftu,BTC
4,huyenhoang.design,BTC


### TODO#1: Requirements
Tự trả lời các câu hỏi sau, từ góc nhìn của bạn (đặt mình vào vị trí bạn là co-founder của dự án DataCracy): 
1. Mục đích của lớp học Atom là gì?
2. BTC sẽ quan tâm đến những chủ đề/quy trình gì để đạt được Mục Đích trong (1)?
3. Làm sao để đo lường các điểm trong (2)? => Metrics?
4. Dựa vào các data đã có như liệt kê trong `STEP 0`:
  * Chỉ dùng những data sẵn có, ta có thể đo lường và thiết kế những metrics nào bạn đã liệt kê trong (3)?
  * Tham khảo Slack API và hình dung về các thông tin DataCracy có khả năng thu thập, bạn sẽ đề xuất DataCracy thu thập thêm những thông tin gì?


In [ ]:
## Điền vào bên dưới câu trả lời của bạn
''' 
REQUIREMENTS
---
1. Mục đích lớp học: rèn luyện cách tự học, giới thiệu sơ lược các công cụ phân tích và tư duy về dữ liệu
2. Mức độ hoàn thành bài tập, khả năng hoàn thành một dự án được giao
3. Số lượng nộp bài tập, thời gian làm bài tập, chất lượng bài hoàn thành, tự đánh giá mức độ nâng cao kiến thức/kĩ năng qua từng bài học + giai đoạn
4. Các chỉ tiêu có thể đo lường:
- tỉ lệ nộp bài: nộp đúng hạn + nộp đủ
thực hiện khảo sát (pool hoặc ks khác) thêm sau mỗi buổi học/assignment về:
- khảo sát thời gian hoàn thành ở mỗi bài tập
- đánh giá độ khó
- đánh giá độ hứng thú
- điểm bạn học được qua bài tập là gì?
- điểm mà bạn còn thấy khó khăn và cần hỗ trợ thêm?
'''

## STEP 2: TỔ CHỨC THÔNG TIN
> Thu thập và hệ thống lại các thông tin

In [318]:
## Hints: info() để check các thông tin (Column), số dòng (Count), và Data Type của mỗi cột
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       74 non-null     object
 1   name          74 non-null     object
 2   display_name  49 non-null     object
 3   real_name     74 non-null     object
 4   title         5 non-null      object
 5   is_bot        74 non-null     bool  
dtypes: bool(1), object(5)
memory usage: 3.1+ KB


In [15]:
channel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 8 columns):
channel_id      19 non-null object
channel_name    19 non-null object
is_channel      19 non-null bool
creator         19 non-null object
created_at      19 non-null datetime64[ns]
topics          6 non-null object
purpose         13 non-null object
num_members     19 non-null int64
dtypes: bool(1), datetime64[ns](1), int64(1), object(5)
memory usage: 1.2+ KB


In [18]:
msg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 8 columns):
channel_id          251 non-null object
msg_id              251 non-null object
msg_ts              251 non-null datetime64[ns]
user_id             251 non-null object
latest_reply        251 non-null datetime64[ns]
reply_user_count    251 non-null int64
reply_users         251 non-null object
github_link         93 non-null object
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 15.8+ KB


## STEP 3: NGUYÊN TẮC CẦN ÁP DỤNG

In [ ]:
## Điền vào bên dưới câu trả lời của bạn
''' 
RULES
---
1. Điền tên đầy đủ vào real name và vị trí (learner/mentor) vào title của Slack
2. Chỉ up link assignment vào đúng channel submit
3. #submit vào cuối bài submit
4. #review vào cuối bài review
5. Đảm bảo khi để link check lại xem có click vào được hay không
'''

In [ ]:
## Điền vào bên dưới câu trả lời của bạn
''' 
LOGICS
---
1. Nên lọc user the trường ID sẽ đảm bảo unique value
2. Ngày lastest reply nếu = 1970 thì thay bằng NULL
3. 
4. 
'''

In [ ]:
## TODO2,3,4,5
## Đặt FK bên cạnh col bạn chọn làm FK
## Và thể hiện mối quan hệ bằng: id >- bảng_gốc.id (FK là PK trong bảng gốc)
''' 
# Dtype obj: object -> string, bool, int, float
# NULL - Nếu có null count < số entries

# Ví dụ  bảng user_df 
user_df
-
user_id string PK
name string
display_name NULL string
real_name string
title NULL string
is_bot bool

channel_df
-
channel_id string PK
channel_name    string
is_channel      bool
creator         string FK >- user_df.user_id
created_at      datetime64[ns]
topics          string
purpose         string
num_members     int

msg_df
-
channel_id          string FK >- channel_df.channel_id
msg_id              string PK
msg_ts              datetime64[ns]
user_id             string FK >- user_df.user_id
latest_reply        datetime64[ns]
reply_user_count    int
reply_users         string
github_link         NULL string

dtc_groups
-
name                string PK FK >- user_df.name
DataCracy_role      string

### Test Database Design by SQL
* Để giúp các bạn làm quen với việc thao tác trên SQL, ta sử dụng thư việc duckdb, cho phép ta thao tác với Data trên Python bằng SQL: https://duckdb.org/2021/05/14/sql-on-pandas.html
* Để hiểu các code SQL bên dưới, tham khảo [SQL Basic Cheatsheet](https://learnsql.com/blog/sql-basics-cheat-sheet/), xem các phần:
  * Querying Single Table
  * Aliases 
  * Filtering
  * Querying Multiple Tables: Các dạng Join
  * Aggregation Functions

* Dựa vào DB Diagram bạn đã vẽ, sẽ giúp bạn dễ dàng theo dõi hơn trong việc join và xử lý data

In [162]:
con = duckdb.connect(database=':memory:', read_only=False) # -> Tạo DB connection
# create a table
## Đoạn code dưới join 2 bảng user_df vằ dtc_groups bằng key name => Tạo thành bảng members_df
SQL_dim_members = '''
SELECT 
  t1.*,
  t2.DataCracy_role
FROM user_df AS t1
JOIN dtc_groups AS t2
ON t1.name = t2.name
'''
members_df=con.execute(SQL_dim_members).fetch_df() # -> Chạy đoạn lệch SQLDataframe

In [166]:
members_df

,user_id,name,display_name,real_name,title,is_bot,DataCracy_role
0,USLACKBOT,slackbot,Slackbot,Slackbot,NaN,False,BTC
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,NaN,False,BTC
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,NaN,False,BTC
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,False,BTC
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,NaN,False,BTC
...,...,...,...,...,...,...,...
60,U020P6WPAQJ,thuynh.386,NaN,Thuy Nguyen,NaN,False,Mentor_Gr4
61,U020P877V88,huongntn2202,NaN,Huong Nguyen Thi Nhi,NaN,False,NaN
62,U021TFMH1RU,mquang.151999,Quang Nguyen,Quang Nguyen,NaN,False,NaN
63,U021Y57RMN1,lamlv723,Lâm Lê,Lam Le,NaN,False,NaN


In [22]:
con.execute("SELECT * FROM channel_df LIMIT 7").fetch_df() # -> In 5 dòng đầu tiên của bảng channel_df ra Dataframe

,channel_id,channel_name,is_channel,creator,created_at,topics,purpose,num_members
0,C01B4PVGLVB,general,True,U01BE2PR6LU,2020-09-23 17:25:03,NaN,This is the one channel that will always inclu...,65
1,C01BYH7JHB5,contents,True,U01BE2PR6LU,2020-10-05 14:28:09,NaN,NaN,12
2,C01CAMNCJJV,branding-design,True,U01AVDY7JET,2020-10-05 10:16:30,NaN,NaN,9
3,C01U6P7LZ8F,atom-assignment1,True,U01BE2PR6LU,2021-04-17 14:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44
4,C01UL6K1C7L,atom-week1,True,U01BE2PR6LU,2021-04-17 00:44:32,TOPIC: Data Strategy. Metrics Design. Spreadsh...,TOPIC: Data Strategy. Metrics Design. Spreadsh...,43
5,C01ULCHGN75,atom-general,True,U01BE2PR6LU,2021-04-17 00:42:20,General communication channel of ATOM,General communication channel of ATOM,24
6,C020VMT58JK,topics-data-analytics,True,U01BE2PR6LU,2021-05-08 13:05:40,NaN,NaN,19


In [17]:
con.execute("SELECT * FROM msg_df LIMIT 5").fetch_df() # -> In 5 dòng đầu tiên của bảng msg_df ra Dataframe

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link
0,C01B4PVGLVB,DEBF1A3C-17AA-4D53-AC27-1FE0884243DB,2021-05-28 20:50:27.000800,U01JQ5U1RPV,2021-05-28 20:51:51.001200,1,NaN,NaN
1,C01B4PVGLVB,48ba219d-fda8-40f2-be65-00735c058071,2021-05-25 11:59:27.000600,U01BE2PR6LU,2021-05-25 13:04:44.006500,4,NaN,NaN
2,C01B4PVGLVB,89919b55-bfe9-4c15-bf45-3021048c7c28,2021-05-24 18:49:08.061100,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,NaN,NaN
3,C01B4PVGLVB,18467584-5810-4bb9-80fe-4b92af9b28d0,2021-05-23 23:26:04.057000,U01UTGT8CVA,2021-05-24 15:36:52.058100,2,NaN,NaN
4,C01B4PVGLVB,dfd2bf1a-69a1-4ac3-aacb-e2daaae8cc04,2021-05-23 14:57:36.055300,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,NaN,NaN


In [18]:
## Đoạn code dưới: Theo từng channel, count số message (phải join với bảng channel_df để lấy tên channel)
SQL = '''
SELECT 
  t2.channel_name,
  COUNT(DISTINCT t1.msg_id) AS msg_cnt
FROM msg_df AS t1
JOIN channel_df AS t2
ON t1.channel_id = t2.channel_id
GROUP BY t2.channel_name 
ORDER BY COUNT(DISTINCT t1.msg_id) DESC
'''
con.execute(SQL).fetch_df()

,channel_name,msg_cnt
0,general,43
1,atom-assignment1,42
2,atom-assignment2,42
3,atom-assignment3,38
4,discuss-group1,32
5,discuss-group3,16
6,topics-python,13
7,discuss-group4,11
8,atom-assignment4,10
9,atom-week1,6


In [20]:
## Đoạn code dưới: Lấy top 3 mentors post nhiều message nhất trong discuss-group của các nhóm
SQL = '''
WITH msg_cnt AS ( ------------- > (1) Chain SQL: tạo bảng tạm thời msg_cnt: count số msg theo user, theo channel
  SELECT 
    user_id,
    channel_id,
    COUNT(msg_id) AS msg_cnt
  FROM msg_df 
  GROUP BY 1, 2
)
SELECT
  t2.real_name,
  t3.channel_name,
  t1.msg_cnt,
  t2.DataCracy_role
FROM msg_cnt AS t1
JOIN members_df AS t2 ------------ > (2) Join msg_count với members_df để lấy tên và role
ON t1.user_id = t2.user_id
JOIN channel_df AS t3 --> (3) Join với channel_df để lấy tên channel
ON t1.channel_id = t3.channel_id
WHERE t2.DataCracy_role LIKE 'Mentor%' ------------- > (4) Filter Mentors
AND t3.channel_name LIKE 'discuss-group%' ---------- > (5) Filter channel discuss theo các group
ORDER BY t1.msg_cnt DESC ---------- > (6) Sẵp xếp theo số msg từ cao xuống thấp
--LIMIT 3 ------------> (7) Lấy top 3
'''
con.execute(SQL).fetch_df()

,real_name,channel_name,msg_cnt,DataCracy_role
0,Phu NDT,discuss-group1,11,Mentor_Gr1
1,Nguyễn Quang Long,discuss-group3,9,Mentor_Gr3
2,Thuy Nguyen,discuss-group4,4,Mentor_Gr4
3,Tan Trần,discuss-group2,2,Mentor_Gr2


### TODO#6: SQL
* Thay đổi các phần trong những đoạn code SQL trên, print kết quả để hiểu về code
* Tham khảo thêm [SQL Basic Cheatsheet](https://learnsql.com/blog/sql-basics-cheat-sheet/)
* Và, GOOGLE! + Cùng trao đổi trên Slack
* Và viết SQL để trả lời các câu hỏi sau:
```
  1. Learners groups nào hoạt động tích cực trên Slack nhất? (tính theo message count)?
  Learners nào nộp bài sớm nhất trong Assignment 1, 2, 3?
  2. Learner nào nộp bài trễ nhất trong Assignment 1, 2, 3?
  3. Learner nào chưa nộp bài Assignment 3?
  4. Learner nào chưa nộp bất kỳ 1 assignment nào?
  5. Tỷ lệ % Learner đã nộp assignment 1, 2, 3? (*giả sử có message trong channel atom-assignment, được tính là đã submit*)
  6. Tỷ lệ % Learner đã submit bài và dc review trong assignment 1, 2, 3?
  7. Learners theo Group nào có tỷ lệ % hoàn thành bài tập cao nhất?
```
* `[Optional]` Bạn có thể tự đặt thêm bất kỳ câu hỏi nào bạn quan tâm



In [30]:
## Top 3 Learners hoạt động tích cực trên Slack nhất (tính theo message count)
SQL_1 = '''
WITH msg_cnt AS ( ------------- > (1) Chain SQL: tạo bảng tạm thời msg_cnt: count số msg theo user, theo channel
  SELECT 
    user_id,
    channel_id,
    COUNT(msg_id) AS msg_cnt
  FROM msg_df 
  GROUP BY 1, 2
)
SELECT
  t2.real_name,
  t1.msg_cnt,
  t2.DataCracy_role
FROM msg_cnt AS t1
JOIN members_df AS t2 ------------ > (2) Join msg_count với members_df để lấy tên và role
ON t1.user_id = t2.user_id
WHERE t2.DataCracy_role LIKE 'Learner%' ------------- > (4) Filter Learner
ORDER BY t1.msg_cnt DESC ---------- > (6) Sẵp xếp theo số msg từ cao xuống thấp
LIMIT 3 ------------> (7) Lấy top 3
'''
a=con.execute(SQL_1).fetch_df()

In [34]:
a

,real_name,msg_cnt,DataCracy_role
0,Việt Hồng Chu,11,Learner_Gr1
1,Hanh Nguyen,4,Learner_Gr4
2,Vy Le,3,Learner_Gr4


In [144]:
#Bang the hien thoi gian gui tin nhan trong cac group nop bai
SQL_1a = '''
-- YOUR CODE
SELECT
  t2.user_id,  
  t2.real_name,
  t3.channel_name,
  t1.msg_ts,
  t1.github_link
FROM msg_df AS t1
JOIN members_df AS t2 ------------ > (2) Join msg_count với members_df để lấy tên và role
ON t1.user_id = t2.user_id
JOIN channel_df AS t3 --> (3) Join với channel_df để lấy tên channel
ON t1.channel_id = t3.channel_id
WHERE t2.DataCracy_role LIKE 'Learner%' ------------- > (4) Filter Learner
AND t3.channel_name LIKE 'atom-assignment%' ---------- > (5) Filter channel discuss theo các group
AND t1.github_link IS NOT NULL ---------- > (5) Filter tin nhan co link submission
ORDER BY t1.msg_ts  ---------- > (6) Sẵp xếp theo thời gian từ cao xuống thấp
'''
submit = con.execute(SQL_1a).fetch_df()

In [145]:
submit.tail()

,user_id,real_name,channel_name,msg_ts,github_link
89,U01VB632FPS,Anh Ha Van Huỳnh,atom-assignment4,2021-05-26 22:03:21.006900,https://github.com/Ha-Huynh-Anh/
90,U01U6JQB695,Giang Phan,atom-assignment3,2021-05-27 11:51:30.001100,https://github.com/Tenderriver/
91,U01USGKQ771,Ngo Thi Minh Trang,atom-assignment3,2021-05-27 22:30:16.003200,https://github.com/TrangNgo1709/
92,U01UMJ36QD9,Quan Nguyễn,atom-assignment4,2021-05-28 00:03:22.002600,https://github.com/Quan030994/
93,U01UMC0N5U2,Việt Hồng Chu,atom-assignment4,2021-05-28 19:53:25.001600,https://github.com/chuviethongfpt/


In [136]:
##Thời gian nộp bài sớm nhât + trễ nhất của các assignment
SQL_1b = '''
    SELECT
        channel_name,
        MIN(msg_ts) AS soonest_submit, 
        MAX(msg_ts) AS lastest_submit
    FROM submit
    GROUP BY channel_name
'''
sb = con.execute(SQL_1b).fetch_df()


In [137]:
sb

,channel_name,soonest_submit,lastest_submit
0,atom-assignment1,2021-04-21 13:00:34.000800,2021-05-10 11:51:46.004700
1,atom-assignment2,2021-05-12 22:50:48.004000,2021-05-22 12:39:34.015500
2,atom-assignment3,2021-05-16 19:08:08.018900,2021-05-27 22:30:16.003200
3,atom-assignment4,2021-05-26 00:46:52.015200,2021-05-28 19:53:25.001600


In [146]:
##Learners nào nộp bài sớm nhất trong Assignment 1, 2, 3?
SQL_t = '''
SELECT
  t.real_name,
  t.channel_name,
  t.msg_ts,
  t.github_link
FROM submit as t
WHERE t.msg_ts = (
  SELECT 
  soonest_submit
  FROM sb
  WHERE channel_name = t.channel_name
)
'''
con.execute(SQL_t).fetch_df()

,real_name,channel_name,msg_ts,github_link
0,Hưng Dương Phan Nam,atom-assignment1,2021-04-21 13:00:34.000800,https://github.com/duongphannamhung/
1,Toan Tran,atom-assignment2,2021-05-12 22:50:48.004000,https://github.com/toantran2206/
2,Hanh Nguyen,atom-assignment3,2021-05-16 19:08:08.018900,https://github.com/nguyenhonghanh2511/
3,Cao Nguyen Huy Hoang,atom-assignment4,2021-05-26 00:46:52.015200,https://github.com/cnhhoang850/


In [143]:
##Learners nào nộp bài trễ nhất trong Assignment 1, 2, 3?
SQL_l = '''
SELECT
  t.real_name,
  t.channel_name,
  t.msg_ts,
  t.github_link
FROM submit as t
WHERE t.msg_ts = (
  SELECT 
  lastest_submit
  FROM sb
  WHERE channel_name = t.channel_name
)
'''
con.execute(SQL_l).fetch_df()

,real_name,channel_name,msg_ts,github_link
0,Danh Phan,atom-assignment1,2021-05-10 11:51:46.004700,https://github.com/danhpcv/
1,Hanh Nguyen,atom-assignment2,2021-05-22 12:39:34.015500,https://github.com/nguyenhonghanh2511/
2,Ngo Thi Minh Trang,atom-assignment3,2021-05-27 22:30:16.003200,https://github.com/TrangNgo1709/
3,Việt Hồng Chu,atom-assignment4,2021-05-28 19:53:25.001600,https://github.com/chuviethongfpt/


In [167]:
members_df.tail()

,user_id,name,display_name,real_name,title,is_bot,DataCracy_role
60,U020P6WPAQJ,thuynh.386,NaN,Thuy Nguyen,NaN,False,Mentor_Gr4
61,U020P877V88,huongntn2202,NaN,Huong Nguyen Thi Nhi,NaN,False,NaN
62,U021TFMH1RU,mquang.151999,Quang Nguyen,Quang Nguyen,NaN,False,NaN
63,U021Y57RMN1,lamlv723,Lâm Lê,Lam Le,NaN,False,NaN
64,U0233KHLM2M,vuthanhdat.contact,NaN,Vuthanhdat Contact,NaN,False,Learner_Gr4


In [172]:
###3Learner nào chưa nộp bài Assignment 3
SQL3 = '''
SELECT
  t.real_name,  
  t.DataCracy_role
FROM members_df as t
WHERE t.user_id NOT IN (
  SELECT 
  user_id
  FROM submit
  WHERE channel_name = 'atom-assignment3'
)
AND t.DataCracy_role LIKE 'Learner%'
'''
con.execute(SQL3).fetch_df()

,real_name,DataCracy_role
0,Viet Hoang,Learner_Gr4
1,Ngoc Huyen,Learner_Gr1
2,Trang Nguyễn,Learner_Gr3
3,Nguyen Thi Kim Minh,Learner_Gr2
4,Nguyen Linh,Learner_Gr1
5,Minh-Thu Nguyen Hoang,Learner_Gr4
6,Việt Hồng Chu,Learner_Gr1
7,Tran Pham,Learner_Gr3
8,Linh Vu,Learner_Gr4
9,anh thu nguyen huynh,Learner_Gr4


In [171]:
###4Learner nào chưa nộp bài cả 3 bài
SQL4 = '''
SELECT
  t.real_name,  
  t.DataCracy_role
FROM members_df as t
WHERE t.user_id NOT IN (
  SELECT 
  user_id
  FROM submit
)
AND t.DataCracy_role LIKE 'Learner%'
'''
con.execute(SQL4).fetch_df()

,real_name,DataCracy_role
0,Ngoc Huyen,Learner_Gr1
1,Nguyen Linh,Learner_Gr1
2,Gia Hoang Tran,Learner_Gr1
3,Vuthanhdat Contact,Learner_Gr4


  5. Tỷ lệ % Learner đã nộp assignment 1, 2, 3? (*giả sử có message trong channel atom-assignment, được tính là đã submit*)
  6. Tỷ lệ % Learner đã submit bài và dc review trong assignment 1, 2, 3?
  7. Learners theo Group nào có tỷ lệ % hoàn thành bài tập cao nhất?

## Pandas vs. SQL 
* Python Pandas và SQL dù là 2 ngôn ngữ khác nhau, nhưng về cách thức thao tác và chuyển đổi data thì như nhau
* Tuỳ theo từng tình huống cụ thể mà ta sẽ sử dụng Python Pandas hay SQL
* Nhưng các bước thao tác/khám phá/tổng hợp data căn bản nhất gồm:

| Thao tác   |     SQL      |  Python | SpreadSheet |
|------------|-------------|------|----------|
| Filter/Selection |  WHERE | df['col'] | Filter |
| Join Data | JOIN | .join() | - |
| Group Data |    GROUP BY   |   .groupby(col) | Pivot |
| Summarize | SUM, AVG, MIN, MAX | .sum(), .mean(), .min(), .max() | SUM, MIN, MAX |

* Tìm hiểu cách thao tác bằng Pandas. [Pandas CheatSheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) => Xem các phần:
  * **Subset(Rows & Columns)**
  * **Summarize Data**: Để làm các phép tổng hợp, thống kê
  * **Group Data**: Tương đương với Pivot trong Excel và `GROUP BY` trong SQL
  * **Combine Data**: Tương đương với `JOIN` trong SQL
  * **Plot**


In [284]:
## Ví dụ: Đoạn code (Pandas) sau làm
## 1. Group by channel_id
## 2. Count các message
## 3. Sắp xếp theo thứ tự số message từ cao xuống thấp (ascending=False)
## 4. Filter lấy Top 5
msg_df.groupby('channel_id')['msg_id'].count().sort_values(ascending=False).head(5)

channel_id
C021FSDN7LJ    42
C01U6P7LZ8F    42
C01B4PVGLVB    37
C0227A51SAY    32
C0220KU9399    18
Name: msg_id, dtype: int64

In [283]:
## Kết quả tương ứng bằng SQL
SQL = '''
  SELECT 
    channel_id,
    COUNT(msg_id) AS msg_cnt
  FROM msg_df 
  GROUP BY 1
  ORDER BY COUNT(msg_id) DESC
  LIMIT 5
'''
con.execute(SQL).fetch_df()

,channel_id,msg_cnt
0,C021FSDN7LJ,42
1,C01U6P7LZ8F,42
2,C01B4PVGLVB,37
3,C0227A51SAY,32
4,C0220KU9399,18


### TODO#7 (Optional): Pandas
* Thực hiện lại các thao tác trong `TODO#6.2` bằng Python Pandas

In [285]:
# YOUR CODE